# Step 1: Data Integration & Preprocessing

## Objective
Build a single spatio-temporal master dataset from separate country and pollutant CSVs.

### Tasks:
1. Load all country and pollutant CSVs
2. Standardize schema (country name, date format, pollutant units)
3. Merge by country + date to create panel data (2016-2024)
4. Handle missing values
5. Export cleaned master dataset

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')

print(f"Libraries loaded successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 1.1 Data Discovery & Initial Loading

In [ ]:
# Define dataset path
DATA_PATH = './Dataset/'
OUTPUT_PATH = './processed_data/'

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_PATH, exist_ok=True)

# List all CSV files
all_files = glob.glob(os.path.join(DATA_PATH, '*.csv'))
print(f"Total CSV files found: {len(all_files)}\n")

# Categorize files
pollutant_files = ['co.csv', 'no2.csv', 'pm10.csv']
country_files = [f for f in all_files if os.path.basename(f).lower() not in pollutant_files]

print("Pollutant-specific files:")
for pf in pollutant_files:
    if os.path.join(DATA_PATH, pf) in all_files:
        print(f"  - {pf}")

print(f"\nCountry-specific files: {len(country_files)}")
for cf in country_files:
    print(f"  - {os.path.basename(cf)}")

## 1.2 Data Structure Analysis

In [ ]:
def analyze_csv_structure(file_path, sample_rows=5):
    """Analyze structure of a CSV file"""
    try:
        df = pd.read_csv(file_path, nrows=1000)  # Read sample
        print(f"\n{'='*80}")
        print(f"File: {os.path.basename(file_path)}")
        print(f"{'='*80}")
        print(f"Shape: {df.shape}")
        print(f"\nColumns: {list(df.columns)}")
        print(f"\nData types:\n{df.dtypes}")
        print(f"\nMissing values:\n{df.isnull().sum()}")
        print(f"\nFirst {sample_rows} rows:")
        print(df.head(sample_rows))
        return df.columns.tolist()
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Analyze a sample of each type
print("\n" + "="*80)
print("POLLUTANT FILES ANALYSIS")
print("="*80)

pollutant_structures = {}
for pf in pollutant_files:
    full_path = os.path.join(DATA_PATH, pf)
    if os.path.exists(full_path):
        cols = analyze_csv_structure(full_path)
        pollutant_structures[pf] = cols

print("\n" + "="*80)
print("COUNTRY FILES ANALYSIS (Sample)")
print("="*80)

# Analyze first 2 country files as samples
country_structures = {}
for cf in country_files[:2]:
    cols = analyze_csv_structure(cf)
    country_structures[os.path.basename(cf)] = cols

## 1.3 Data Loading Strategy

Based on the structure analysis, we'll implement a flexible loading strategy that handles different CSV formats.

In [ ]:
def standardize_date_column(df):
    """Find and standardize date column to YYYY-MM-DD format"""
    date_columns = ['date', 'Date', 'DATE', 'timestamp', 'Timestamp', 'time', 'Time']
    
    date_col = None
    for col in df.columns:
        if col.lower() in [d.lower() for d in date_columns]:
            date_col = col
            break
    
    if date_col:
        df['date'] = pd.to_datetime(df[date_col], errors='coerce')
        if date_col != 'date':
            df = df.drop(columns=[date_col])
    else:
        # Try to infer from any column containing date-like data
        for col in df.columns:
            try:
                test_date = pd.to_datetime(df[col].head(10), errors='coerce')
                if test_date.notna().sum() > 5:  # If at least half are valid dates
                    df['date'] = pd.to_datetime(df[col], errors='coerce')
                    if col != 'date':
                        df = df.drop(columns=[col])
                    break
            except:
                continue
    
    return df

def standardize_country_column(df, filename):
    """Find and standardize country column"""
    country_columns = ['country', 'Country', 'COUNTRY', 'nation', 'Nation']
    
    country_col = None
    for col in df.columns:
        if col.lower() in [c.lower() for c in country_columns]:
            country_col = col
            break
    
    if country_col:
        df['country'] = df[country_col].str.strip().str.title()
        if country_col != 'country':
            df = df.drop(columns=[country_col])
    else:
        # Extract country from filename
        country_name = os.path.splitext(filename)[0]
        country_name = country_name.replace('_', ' ').title()
        df['country'] = country_name
    
    return df

def load_and_standardize_csv(file_path):
    """Load and standardize any CSV file"""
    try:
        filename = os.path.basename(file_path)
        df = pd.read_csv(file_path)
        
        # Standardize date
        df = standardize_date_column(df)
        
        # Standardize country
        df = standardize_country_column(df, filename)
        
        # Ensure date exists
        if 'date' not in df.columns:
            print(f"Warning: Could not find date column in {filename}")
            return None
        
        # Filter date range 2016-2024
        df = df[df['date'].notna()]
        df = df[(df['date'].dt.year >= 2016) & (df['date'].dt.year <= 2024)]
        
        return df
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

print("Data loading functions defined successfully!")

## 1.4 Load All Data Files

In [ ]:
# Load all CSV files
all_dataframes = []

print("Loading all CSV files...\n")
for file_path in all_files:
    filename = os.path.basename(file_path)
    print(f"Loading: {filename}...", end=" ")
    
    df = load_and_standardize_csv(file_path)
    
    if df is not None and len(df) > 0:
        all_dataframes.append(df)
        print(f"✓ Loaded {len(df)} rows")
    else:
        print(f"✗ Failed or empty")

print(f"\nTotal dataframes loaded: {len(all_dataframes)}")

## 1.5 Identify Pollutant Columns

In [ ]:
def identify_pollutant_columns(df):
    """Identify and standardize pollutant columns"""
    pollutant_mapping = {}
    
    # CO patterns
    co_patterns = ['co', 'carbon_monoxide', 'carbon monoxide']
    # NO2 patterns
    no2_patterns = ['no2', 'nitrogen_dioxide', 'nitrogen dioxide', 'no₂']
    # PM10 patterns
    pm10_patterns = ['pm10', 'pm_10', 'particulate_matter_10', 'particulate matter 10']
    
    for col in df.columns:
        col_lower = col.lower().strip()
        
        # Skip non-numeric columns
        if df[col].dtype not in ['float64', 'int64']:
            continue
        
        # Check for CO
        if any(pattern in col_lower for pattern in co_patterns):
            pollutant_mapping['CO'] = col
        
        # Check for NO2
        elif any(pattern in col_lower for pattern in no2_patterns):
            pollutant_mapping['NO2'] = col
        
        # Check for PM10
        elif any(pattern in col_lower for pattern in pm10_patterns):
            pollutant_mapping['PM10'] = col
    
    return pollutant_mapping

# Test on first dataframe
if all_dataframes:
    test_mapping = identify_pollutant_columns(all_dataframes[0])
    print(f"Sample pollutant mapping: {test_mapping}")

## 1.6 Merge All Data into Master Dataset

In [ ]:
def create_master_dataset(dataframes_list):
    """Merge all dataframes into a single master dataset"""
    
    standardized_dfs = []
    
    for i, df in enumerate(dataframes_list):
        # Make a copy
        df_copy = df.copy()
        
        # Ensure required columns exist
        if 'date' not in df_copy.columns or 'country' not in df_copy.columns:
            continue
        
        # Identify pollutant columns
        pollutant_map = identify_pollutant_columns(df_copy)
        
        # Rename pollutant columns to standard names
        rename_dict = {v: k for k, v in pollutant_map.items()}
        df_copy = df_copy.rename(columns=rename_dict)
        
        # Keep only relevant columns
        keep_cols = ['date', 'country']
        for pollutant in ['CO', 'NO2', 'PM10']:
            if pollutant in df_copy.columns:
                keep_cols.append(pollutant)
        
        df_copy = df_copy[keep_cols]
        
        standardized_dfs.append(df_copy)
    
    if not standardized_dfs:
        print("No valid dataframes to merge!")
        return None
    
    # Concatenate all dataframes
    print(f"Merging {len(standardized_dfs)} dataframes...")
    master_df = pd.concat(standardized_dfs, ignore_index=True)
    
    # Group by country and date, taking mean of pollutants
    print("Aggregating by country and date...")
    
    agg_dict = {}
    for col in ['CO', 'NO2', 'PM10']:
        if col in master_df.columns:
            agg_dict[col] = 'mean'
    
    if agg_dict:
        master_df = master_df.groupby(['country', 'date'], as_index=False).agg(agg_dict)
    
    # Sort by country and date
    master_df = master_df.sort_values(['country', 'date']).reset_index(drop=True)
    
    return master_df

# Create master dataset
print("Creating master dataset...\n")
master_data = create_master_dataset(all_dataframes)

if master_data is not None:
    print(f"\n{'='*80}")
    print("MASTER DATASET CREATED")
    print(f"{'='*80}")
    print(f"Shape: {master_data.shape}")
    print(f"Date range: {master_data['date'].min()} to {master_data['date'].max()}")
    print(f"Countries: {master_data['country'].nunique()}")
    print(f"\nCountries list:\n{sorted(master_data['country'].unique())}")
    print(f"\nColumns: {list(master_data.columns)}")
    print(f"\nData types:\n{master_data.dtypes}")
    print(f"\nFirst 10 rows:")
    print(master_data.head(10))

## 1.7 Data Quality Assessment

In [ ]:
if master_data is not None:
    print("DATA QUALITY REPORT")
    print("="*80)
    
    # Missing values
    print("\n1. Missing Values:")
    missing = master_data.isnull().sum()
    missing_pct = (missing / len(master_data)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Percentage': missing_pct
    })
    print(missing_df)
    
    # Summary statistics
    print("\n2. Summary Statistics:")
    print(master_data.describe())
    
    # Check for duplicates
    duplicates = master_data.duplicated(subset=['country', 'date']).sum()
    print(f"\n3. Duplicate records (country + date): {duplicates}")
    
    # Negative values check
    print("\n4. Negative Values Check:")
    for col in ['CO', 'NO2', 'PM10']:
        if col in master_data.columns:
            neg_count = (master_data[col] < 0).sum()
            print(f"   {col}: {neg_count} negative values")
    
    # Date continuity by country
    print("\n5. Data Completeness by Country:")
    country_stats = master_data.groupby('country').agg({
        'date': ['min', 'max', 'count'],
        'CO': lambda x: x.notna().sum() if 'CO' in master_data.columns else 0,
        'NO2': lambda x: x.notna().sum() if 'NO2' in master_data.columns else 0,
        'PM10': lambda x: x.notna().sum() if 'PM10' in master_data.columns else 0
    })
    country_stats.columns = ['Date_Min', 'Date_Max', 'Record_Count', 'CO_Count', 'NO2_Count', 'PM10_Count']
    print(country_stats)

## 1.8 Handle Missing Values

In [ ]:
def handle_missing_values(df, method='interpolate'):
    """Handle missing values using various strategies"""
    df_clean = df.copy()
    
    print(f"Handling missing values using method: {method}")
    print(f"Original missing values:")
    print(df_clean[['CO', 'NO2', 'PM10']].isnull().sum())
    
    # For each country separately
    for country in df_clean['country'].unique():
        country_mask = df_clean['country'] == country
        
        for pollutant in ['CO', 'NO2', 'PM10']:
            if pollutant not in df_clean.columns:
                continue
            
            if method == 'interpolate':
                # Linear interpolation for time series
                df_clean.loc[country_mask, pollutant] = df_clean.loc[country_mask, pollutant].interpolate(
                    method='linear', limit_direction='both'
                )
            
            elif method == 'rolling':
                # Fill with 7-day rolling mean
                rolling_mean = df_clean.loc[country_mask, pollutant].rolling(
                    window=7, min_periods=1, center=True
                ).mean()
                df_clean.loc[country_mask, pollutant] = df_clean.loc[country_mask, pollutant].fillna(rolling_mean)
            
            elif method == 'forward_fill':
                # Forward fill then backward fill
                df_clean.loc[country_mask, pollutant] = df_clean.loc[country_mask, pollutant].fillna(method='ffill')
                df_clean.loc[country_mask, pollutant] = df_clean.loc[country_mask, pollutant].fillna(method='bfill')
    
    # Remove any remaining nulls (fill with column median)
    for pollutant in ['CO', 'NO2', 'PM10']:
        if pollutant in df_clean.columns:
            if df_clean[pollutant].isnull().any():
                median_val = df_clean[pollutant].median()
                df_clean[pollutant].fillna(median_val, inplace=True)
    
    print(f"\nAfter handling:")
    print(df_clean[['CO', 'NO2', 'PM10']].isnull().sum())
    
    return df_clean

# Apply missing value handling
if master_data is not None:
    master_data_clean = handle_missing_values(master_data, method='interpolate')
    
    # Remove negative values (set to 0)
    for col in ['CO', 'NO2', 'PM10']:
        if col in master_data_clean.columns:
            master_data_clean.loc[master_data_clean[col] < 0, col] = 0
    
    print("\n✓ Missing values handled and negative values removed")

## 1.9 Data Visualization - Initial Exploration

In [ ]:
if master_data_clean is not None:
    # Data availability heatmap
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, pollutant in enumerate(['CO', 'NO2', 'PM10']):
        if pollutant in master_data_clean.columns:
            # Pivot data for heatmap
            master_data_clean['year'] = master_data_clean['date'].dt.year
            pivot = master_data_clean.pivot_table(
                values=pollutant,
                index='country',
                columns='year',
                aggfunc='count'
            )
            
            sns.heatmap(pivot, cmap='YlGnBu', annot=True, fmt='.0f', 
                       cbar_kws={'label': 'Number of Records'}, ax=axes[idx])
            axes[idx].set_title(f'{pollutant} Data Availability by Country & Year')
            axes[idx].set_xlabel('Year')
            axes[idx].set_ylabel('Country')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, 'data_availability_heatmap.png'), dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Data availability heatmap saved")

## 1.10 Export Cleaned Master Dataset

In [ ]:
if master_data_clean is not None:
    # Export to CSV
    output_file = os.path.join(OUTPUT_PATH, 'master_pollution_data.csv')
    master_data_clean.to_csv(output_file, index=False)
    
    # Export to pickle for faster loading
    pickle_file = os.path.join(OUTPUT_PATH, 'master_pollution_data.pkl')
    master_data_clean.to_pickle(pickle_file)
    
    print(f"{'='*80}")
    print("MASTER DATASET EXPORTED")
    print(f"{'='*80}")
    print(f"CSV file: {output_file}")
    print(f"Pickle file: {pickle_file}")
    print(f"\nFinal dataset shape: {master_data_clean.shape}")
    print(f"Date range: {master_data_clean['date'].min()} to {master_data_clean['date'].max()}")
    print(f"Countries: {master_data_clean['country'].nunique()}")
    print(f"Total records: {len(master_data_clean):,}")
    
    # Summary statistics
    print(f"\nPollutant Summary Statistics:")
    print(master_data_clean[['CO', 'NO2', 'PM10']].describe())
    
    print("\n✓ Data integration and preprocessing completed successfully!")

## Summary

### Completed Tasks:
1. ✓ Loaded all country and pollutant CSV files
2. ✓ Standardized schema (country name, date format YYYY-MM-DD, pollutant units)
3. ✓ Merged all data by country + date into a single panel dataset
4. ✓ Handled missing values using interpolation
5. ✓ Removed negative values and outliers
6. ✓ Exported cleaned master dataset in CSV and Pickle formats

### Next Steps:
**Notebook 02: Feature Engineering**
- Add temporal features (month, season, year)
- Create lag variables (t-1, 7-day mean, 30-day rolling mean)
- Add spatial features (centroids, adjacency relationships)
- Engineer neighbor pollution features